In [1]:
#@title Insira os dataframes e o resultado será baixado para a sua máquina
from google.colab import files
import io
# Recebe os dados
print("SELECIONE O DATAFRAME DE DADOS DEMOGRÁFICOS: ")
demo = files.upload()
# Pega os dados de dentro do arquivo
demo = list(demo.values())[0]
# Deixa os dados em um formato legível para o pandas
csv_demograficos = io.BytesIO(demo)

# Recebe os dados
print("\n\nSELECIONE O DATAFRAME DE HISTOPATOLIGIA: ")
hist = files.upload()
# Pega os dados de dentro do arquivo
hist = list(hist.values())[0]
# Deixa os dados em um formato legível para o pandas
csv_histopatologia = io.BytesIO(hist)

# Recebe os dados
print("\n\nSELECIONE O DATAFRAME DE PESO E ALTURA: ")
peso = files.upload()
# Pega os dados de dentro do arquivo
peso = list(peso.values())[0]
# Deixa os dados em um formato legível para o pandas
csv_peso_altura = io.BytesIO(peso)

# Recebe os dados
print("\n\nSELECIONE O DATAFRAME DE TUMORES: ")
tum = files.upload()
# Pega os dados de dentro do arquivo
tum = list(tum.values())[0]
# Deixa os dados em um formato legível para o pandas
csv_tumores = io.BytesIO(tum)

import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

!gdown '1aOhfiEo_sAseqTz5UKZuUY5iKH4Wyszv'
!gdown '1bIRe2Xkzxdtk5_5BsKXE5cfl84fT0zBO'
scaler = joblib.load('/content/minMax.scaler')
rf_classifier = joblib.load('/content/modelo_rf_classifier.model')

df_d = pd.read_csv(csv_demograficos)
df_h = pd.read_csv(csv_histopatologia)
df_p = pd.read_csv(csv_peso_altura)
df_t = pd.read_csv(csv_tumores)

def replace_arr(base_str, values_to_replace, replace_str):
  for value in values_to_replace:
    base_str = base_str.replace(value, replace_str)
  return base_str

def normalize_columns_name(df):
  columns = []
  for column in df.columns:
    column = column.lower().strip()
    column = replace_arr(column, [
        '?', '(', ')', '!', 'º', '[', ']', '%', '>', '<', ':', ',', ';', '.', '#', '/' 
    ], '')
    column = replace_arr(column, [
        'á', 'à', 'â', 'ã',
    ], 'a')
    column = replace_arr(column, [
        'é', 'è', 'ê',
    ], 'e')
    column = replace_arr(column, [
        'í', 'ì',
    ], 'i')
    column = replace_arr(column, [
        'ó', 'ò', 'ô', 'õ'
    ], 'o')
    column = replace_arr(column, [
        'ú', 'ù', 'û'
    ], 'u')
    column = replace_arr(column, [
        '-', ' ', '=', ',', '__', '__'
    ], '_')
    column = replace_arr(column, [
        'ç'
    ], 'c')

    if column[-1] == '_':
      column = column[:-1]

    if column[0] == '_':
      column = column[1:]
    
    columns.append(column)

  df.columns = columns
  return df

# Aplica a função que normaliza o nome das colunas
normalize_columns_name(df_d)
# Renomeia as colunas normalizadas
df_d = df_d.rename(columns={
  'tempo_de_seguimento_em_dias_desde_o_ultimo_tumor_no_caso_de_tumores_multiplos_dt_pci': 'dias_desde_ultimo_tumor_ctm_dt_pci',
  'historia_familiar_de_cancer_relacionado_a_sindrome_de_cancer_de_mama_e_ovario_hereditaria_choice_sim_1_grau_apenas_1_caso': 'hered_mama_ovario_grau_1_1_caso',
  'historia_familiar_de_cancer_relacionado_a_sindrome_de_cancer_de_mama_e_ovario_hereditaria_choice_sim_1_grau_mais_de_1_caso': 'hered_mama_ovario_grau_1_n_caso',
  'historia_familiar_de_cancer_relacionado_a_sindrome_de_cancer_de_mama_e_ovario_hereditaria_choice_sim_2_grau_apenas_1_caso': 'hered_mama_ovario_grau_2_1_caso',
  'historia_familiar_de_cancer_relacionado_a_sindrome_de_cancer_de_mama_e_ovario_hereditaria_choice_sim_2_grau_apenas_1_caso': 'hered_mama_ovario_grau_2_n_caso',
  'grau_de_parentesco_de_familiar_com_cancer_choice_primeiro_pais_irmaos_filhos': 'cancer_1_grau_parent',
  'grau_de_parentesco_de_familiar_com_cancer_choice_segundo_avos_tios_e_netos': 'cancer_2_grau_parent',
  'grau_de_parentesco_de_familiar_com_cancer_choice_terceiro_bisavos_tio_avos_primos_sobrinhos': 'cancer_3_grau_parent',
  'idade_do_paciente_ao_primeiro_diagnostico': 'idade_primeiro_diagnostico'
})

# Arruma os nomes das colunas
normalize_columns_name(df_h)
df_h = df_h.rename(columns={
  'receptor_de_progesterona_quantificacao': 'progesterona_qtd',
  'receptorde_estrogenio_quantificacao': 'estrogenio_qtd',
  'ki67': 'ki67_qtd',
})

# Normaliza os nomes das colunas do df peso e altura
normalize_columns_name(df_p)

# Normaliza os nomes das colunas do df de tumores
normalize_columns_name(df_t)

dfd = df_d.loc[:]

# Remove os pacientes que não fizeram adjuvante ou neoadjuvante
dfd = dfd[dfd['regime_de_tratamento'].isin(['Terapia Adjuvante', 'Terapia Neoadjuvante'])]
# Troca NANs pela média
dfd.loc[dfd['idade_primeiro_diagnostico'].isna(), 'idade_primeiro_diagnostico'] = dfd['idade_primeiro_diagnostico'].mean()
# Cria a coluna sobrevida
dfd_sobrevida = dfd

dfd_sobrevida['data_da_cirurgia'] = pd.to_datetime(dfd_sobrevida['data_da_cirurgia'], format='%Y-%m-%d')
dfd_sobrevida['data_da_ultima_informacao_sobre_o_paciente'] = pd.to_datetime(dfd_sobrevida['data_da_ultima_informacao_sobre_o_paciente'], format='%Y-%m-%d')

dfd_sobrevida['sobrevida'] = dfd_sobrevida['data_da_ultima_informacao_sobre_o_paciente'] - dfd_sobrevida['data_da_cirurgia']

dfd_sobrevida['sobrevida'] =dfd_sobrevida['sobrevida'].dt.days / 365.25

dfd['sobrevida'] = dfd_sobrevida['sobrevida']
mean = dfd['sobrevida'].mean()
std = dfd['sobrevida'].std()

random_values = np.random.normal(mean, std, size=dfd['sobrevida'].isna().sum())

# Substitui apenas os valores nulos da coluna 'sobrevida' pelos valores gerados
dfd.loc[dfd['sobrevida'].isna(), 'sobrevida'] = random_values

# Pega as colunas desejadas do df_d
df_d_grouped = dfd[['record_id', 'atividade_fisica', 'regime_de_tratamento', 'idade_primeiro_diagnostico', 'sobrevida', 'ultima_informacao_do_paciente']]

# Remove NANs
dfh = df_h.loc[:]
dfh = dfh.dropna(subset=['diagnostico_primario_tipo_histologico', 'grau_histologico', 'receptor_de_estrogenio', 'receptor_de_progesterona', 'progesterona_qtd', 'estrogenio_qtd', 'indice_h_receptorde_progesterona'], how='all')
dfh = dfh.dropna(subset=['receptor_de_estrogenio', 'receptor_de_progesterona', 'ki67_14',	'progesterona_qtd',	'estrogenio_qtd',	'indice_h_receptorde_progesterona'], how='all')

dfh.loc[((dfh['receptor_de_estrogenio'].isna()) | (dfh['receptor_de_estrogenio'] == 'inconclusivo')) & (dfh['receptor_de_progesterona'] == 'positivo'), 'receptor_de_estrogenio'] = 'positivo'
dfh.loc[((dfh['receptor_de_estrogenio'].isna()) | (dfh['receptor_de_estrogenio'] == 'inconclusivo')) & (dfh['receptor_de_progesterona'] == 'negativo'), 'receptor_de_estrogenio'] = 'negativo'

dfh.loc[((dfh['receptor_de_progesterona'].isna()) | (dfh['receptor_de_progesterona'] == 'inconclusivo')) & (dfh['receptor_de_estrogenio'] == 'positivo'), 'receptor_de_progesterona'] = 'positivo'
dfh.loc[((dfh['receptor_de_progesterona'].isna()) | (dfh['receptor_de_progesterona'] == 'inconclusivo')) & (dfh['receptor_de_estrogenio'] == 'negativo'), 'receptor_de_progesterona'] = 'negativo'

# ESTROGENIO = NEGATIVO - ESTROGENIO QUANTIFICACAO
# As celulas com estrogenio = negativo são células incapazes de receber estrogênio. Portanto, todos os casos em que a quantificação for 'neg' ou 'NEG', troquei por 0
dfh.loc[(dfh['estrogenio_qtd'] == 'neg') & (dfh['receptor_de_estrogenio'] == 'negativo'), 'estrogenio_qtd'] = 0
dfh.loc[(dfh['estrogenio_qtd'] == 'NEG') & (dfh['receptor_de_estrogenio'] == 'negativo'), 'estrogenio_qtd'] = 0

# Os casos onde a quantificação é NaN e o estrogenio é negativo, tratei como 0, que é a imensa maioria dos casos
dfh.loc[(dfh['estrogenio_qtd'].isna()) & (dfh['receptor_de_estrogenio'] == 'negativo'), 'estrogenio_qtd'] = 0

# ESTROGENIO = POSITIVO - ESTROGENIO QUANTIFICACAO
# Os registros estranhos fiz que recebessem a moda (1)
dfh.loc[(dfh['estrogenio_qtd'].isna()) & (dfh['receptor_de_estrogenio'] == 'positivo'), 'estrogenio_qtd'] = 1
dfh.loc[(dfh['estrogenio_qtd'] == '0') & (dfh['receptor_de_estrogenio'] == 'positivo'), 'estrogenio_qtd'] = 1
dfh.loc[(dfh['estrogenio_qtd'] == 'neg') & (dfh['receptor_de_estrogenio'] == 'positivo'), 'estrogenio_qtd'] = 1
# Valor entre 1-10 é a média (5)
dfh.loc[(dfh['estrogenio_qtd'] == '1-10') & (dfh['receptor_de_estrogenio'] == 'positivo'), 'estrogenio_qtd'] = 5

# PROGESTERONA
dfh.loc[(dfh['progesterona_qtd'] == '20-70'), 'progesterona_qtd'] = 45
dfh.loc[(dfh['progesterona_qtd'] == '70%'), 'progesterona_qtd'] = 70
dfh.loc[((dfh['progesterona_qtd'] == 'neg') | (dfh['progesterona_qtd'] == 'NEG') | (dfh['progesterona_qtd'].isna())) & (dfh['receptor_de_progesterona'] == 'negativo'),'progesterona_qtd'] = 0
dfh.loc[((dfh['progesterona_qtd'] == 'neg') | (dfh['progesterona_qtd'] == 'inconclusivo') | (dfh['progesterona_qtd'].isna())) & (dfh['receptor_de_progesterona'] == 'positivo'),'progesterona_qtd'] = 1

dfh = dfh.drop(dfh.loc[dfh['receptor_de_estrogenio'] == 'não realizado'].index)

# ki67 - Preenche NaNs de acordo com a média em 4 grupos considerando as variáveis estrogenio e progesterona (ambos positivo ou negativo)
dfh.loc[(dfh['receptor_de_estrogenio'] == 'positivo') & (dfh['receptor_de_progesterona'] == 'negativo') & (dfh['ki67_qtd'].isna()), 'ki67_qtd'] = dfh[(dfh['receptor_de_estrogenio'] == 'positivo') & (dfh['receptor_de_progesterona'] == 'negativo')]['ki67_qtd'].mean()
dfh.loc[(dfh['receptor_de_estrogenio'] == 'negativo') & (dfh['receptor_de_progesterona'] == 'positivo') & (dfh['ki67_qtd'].isna()), 'ki67_qtd'] = dfh[(dfh['receptor_de_estrogenio'] == 'negativo') & (dfh['receptor_de_progesterona'] == 'positivo')]['ki67_qtd'].mean()
dfh.loc[(dfh['receptor_de_estrogenio'] == 'positivo') & (dfh['receptor_de_progesterona'] == 'positivo') & (dfh['ki67_qtd'].isna()), 'ki67_qtd'] = dfh[(dfh['receptor_de_estrogenio'] == 'positivo') & (dfh['receptor_de_progesterona'] == 'positivo')]['ki67_qtd'].mean()
dfh.loc[(dfh['receptor_de_estrogenio'] == 'negativo') & (dfh['receptor_de_progesterona'] == 'negativo') & (dfh['ki67_qtd'].isna()), 'ki67_qtd'] = dfh[(dfh['receptor_de_estrogenio'] == 'negativo') & (dfh['receptor_de_progesterona'] == 'negativo')]['ki67_qtd'].mean()

dfh['ki67_14'] = dfh['ki67_14'].replace({
    'não realizado': np.nan,
    'inconclusivo': np.nan
})

dfh.loc[(dfh['ki67_14'].isna()) & (dfh['ki67_qtd'] >= 20), 'ki67_14'] = 'positivo'
dfh.loc[dfh['ki67_14'].isna(), 'ki67_14'] = 'negativo'

dfh['estrogenio_qtd'] = dfh['estrogenio_qtd'].astype(float)
dfh['progesterona_qtd'] = dfh['progesterona_qtd'].astype(float)
dfh['ki67_qtd'] = dfh['ki67_qtd'].astype(float)

# Agrupando e pegando o primeiro registro de cada RECORD_ID
dfh = dfh.loc[dfh.groupby('record_id')['repeat_instance'].idxmin()]

# tirando os nulos finais
dfh = dfh[(dfh['subtipo_tumoral'].notna()) & (dfh['her2_por_ihc'].notna())]

# Pegando colunas que serão usadas
df_h_grouped = dfh.loc[:, ['record_id', 'subtipo_tumoral', 'receptor_de_estrogenio', 'estrogenio_qtd', 'receptor_de_progesterona', 'progesterona_qtd', 'ki67_14', 'ki67_qtd', 'her2_por_ihc']]
# Ordena os valores por data
df_p = df_p.sort_values(['record_id', 'data'])

# Substitui zeros para serem tratados como nan
df_p['altura_em_centimetros'] = df_p['altura_em_centimetros'].replace({0: np.nan})
df_p['peso'] = df_p['peso'].replace({0: np.nan})

# Agrupa as linhas em uma só
df_p_2 = df_p.groupby(by='record_id').agg({
    'peso': lambda x: x.iloc[0] if not np.isnan(x.iloc[0]) else x.fillna(0).mean(),
    'altura_em_centimetros': lambda x: x.iloc[0] if not np.isnan(x.iloc[0]) else x.fillna(0).median(),
}).reset_index()

# Tira as linhas com algum 0
df_p_2 = df_p_2[(df_p_2['peso'] != 0) & (df_p_2['altura_em_centimetros'] != 0) & (df_p_2['peso'].notna()) & (df_p_2['altura_em_centimetros'].notna())]

# Calcula o IMC
df_p_2['imc'] = df_p_2['peso'] / ((df_p_2['altura_em_centimetros'] / 100) * (df_p_2['altura_em_centimetros'] / 100))

df_p_grouped = df_p_2.loc[:, ['record_id', 'imc']]

# Tirando linhas de NaNs
df_t_2 = df_t.dropna(subset=df_t.columns[1:], how='all')
# Pegando as colunas interessantes
df_t_2 = df_t_2[['record_id', 'repeat_instance', 'estadio_clinico', 'classificacao_tnm_clinico_t', 'classificacao_tnm_clinico_n', 'classificacao_tnm_clinico_m', 'lateralidade_do_tumor', 'com_recidiva_a_distancia', 'com_recidiva_local', 'com_recidiva_regional', 'tempo_desde_o_diagnostico_ate_a_primeira_recidiva']].loc[:]


# Agrupando classificação T
df_t_2['classificacao_tnm_clinico_t'] = df_t_2['classificacao_tnm_clinico_t'].replace({
    '1MIC': '1', # 1
    '1A': '1',
    '1B': '1',
    '1C': '1',
    '2A': '2', # 2
    '2B': '2',
    '3A': '3', # 3
    '3B': '3',
    '4C': '4', # 4
    '4A': '4',
    '4B': '4',
    '4D': '4',
    'CDIS': 'IS', # IS
    'CLIS': 'IS'
})

# Transformando A em 0
df_t_2 = df_t_2[~df_t_2['classificacao_tnm_clinico_t'].isin(['A', '0', 'Y: Na', 'X - nao foi possivel determinar'])]

# Agrupando classificação N
df_t_2['classificacao_tnm_clinico_n'] = df_t_2['classificacao_tnm_clinico_n'].replace({
    '1MIC': '1', # 1
    '1A': '1',
    '1B': '1',
    '1C': '1',
    '2A': '2', # 2
    '2B': '2',
    '3A': '3', # 3
    '3B': '3',
    '3C': '3',
    '4C': '4', # 4
    '4A': '4',
    '4B': '4',
    '4D': '4'
})

# Agrupando classificação M
df_t_2['classificacao_tnm_clinico_m'] = df_t_2['classificacao_tnm_clinico_m'].replace({
    '1MIC': '1', # 1
    '1A': '1',
    '1B': '1',
    '1C': '1',
    '2A': '2', # 2
    '2B': '2',
    '3A': '3', # 3
    '3B': '3',
    '3C': '3',
    '4C': '4', # 4
    '4A': '4',
    '4B': '4',
    '4D': '4'
})

# Pegando o primeiro registro de cada record_id
df_t_2 = df_t_2.loc[df_t_2.groupby('record_id')['repeat_instance'].idxmin()]

# Marcando recidivas
df_t_2['recidiva'] = np.nan

df_t_2.loc[(df_t_2['com_recidiva_a_distancia'] == 'Sim') | (df_t_2['com_recidiva_local'] == 'Sim') | (df_t_2['com_recidiva_regional'] == 'Sim'), 'recidiva'] = 'Sim'
df_t_2.loc[(df_t_2['com_recidiva_a_distancia'] == 'Não') & (df_t_2['com_recidiva_local'] == 'Não') & (df_t_2['com_recidiva_regional'] == 'Não'), 'recidiva'] = 'Não'

df_t_2.loc[df_t_2['tempo_desde_o_diagnostico_ate_a_primeira_recidiva'].isin([np.inf, -np.inf]), 'tempo_desde_o_diagnostico_ate_a_primeira_recidiva'] = np.nan
df_t_2.loc[df_t_2['tempo_desde_o_diagnostico_ate_a_primeira_recidiva'].isna(), 'tempo_desde_o_diagnostico_ate_a_primeira_recidiva'] = 0
df_t_2['tempo_desde_o_diagnostico_ate_a_primeira_recidiva'] = df_t_2['tempo_desde_o_diagnostico_ate_a_primeira_recidiva'].astype(int)
# Guardando resultado final
df_t_grouped = df_t_2.loc[:, ['record_id', 'estadio_clinico', 'classificacao_tnm_clinico_t', 'classificacao_tnm_clinico_n', 'classificacao_tnm_clinico_m', 'tempo_desde_o_diagnostico_ate_a_primeira_recidiva', 'recidiva']]

# Realiza a junção das tabelas baseando-se nos record ids
df = pd.merge(
    df_t_grouped,
    df_p_grouped,
    on='record_id'
)

df = pd.merge(
    df,
    df_h_grouped,
    on='record_id'
)

df = pd.merge(
  df,
  df_d_grouped,
  on='record_id'
)

# cria uma cópia do DataFrame original
dfteste = df.copy()

# remove casos de fracasso que contêm a string "Obito por câncer" na coluna "ultima_informacao_do_paciente"
dfteste = dfteste.query('ultima_informacao_do_paciente != "Obito por câncer"')

# cria uma nova coluna "grupo_de_idade" que armazena o grupo de idade (10 anos de intervalo) de cada paciente
dfteste['grupo_de_idade'] = dfteste['idade_primeiro_diagnostico'].apply(lambda x: int(x/10))

# itera sobre cada grupo de idade e subtipo tumoral
for grupo_idade in dfteste['grupo_de_idade'].unique():
  for subtipo_tumoral in dfteste['subtipo_tumoral'].unique():
    # calcula o primeiro quartil dos tempos desde o diagnóstico até a primeira recidiva para o grupo de idade e subtipo tumoral atual
    # e armazena o valor em "primeiro_quartil"
    primeiro_quartil = dfteste.query('tempo_desde_o_diagnostico_ate_a_primeira_recidiva > 0 & grupo_de_idade == ' + str(grupo_idade) + ' & subtipo_tumoral == ' + str(subtipo_tumoral))['tempo_desde_o_diagnostico_ate_a_primeira_recidiva'].quantile(0.10)
    # verifica se "primeiro_quartil" não é NaN
    if not np.isnan(primeiro_quartil):
      # seleciona os pacientes que estão no grupo de idade e subtipo tumoral atual e que tiveram uma recidiva depois do primeiro quartil de tempo
      pacientes_fracasso = dfteste.query('tempo_desde_o_diagnostico_ate_a_primeira_recidiva > 0 & tempo_desde_o_diagnostico_ate_a_primeira_recidiva > ' + str(primeiro_quartil) + ' & grupo_de_idade == ' + str(grupo_idade) + ' & subtipo_tumoral ==' + str(subtipo_tumoral))
      # remove os pacientes selecionados do DataFrame
      dfteste = dfteste.drop(pacientes_fracasso.index)

# atualiza o DataFrame original com o DataFrame modificado
df = dfteste

# remove colunas
df = df.drop(columns=['recidiva', 'sobrevida', 'grupo_de_idade', 'tempo_desde_o_diagnostico_ate_a_primeira_recidiva'])
df = df.drop(columns=['ultima_informacao_do_paciente'])

# cria uma lista de colunas numéricas a serem selecionadas
colunas_numericas = ['imc', 'estrogenio_qtd', 'progesterona_qtd', 'ki67_qtd', 'idade_primeiro_diagnostico']
# cria um novo DataFrame com apenas as colunas numéricas especificadas
df_numericas = df[colunas_numericas]
# cria um novo DataFrame com todas as colunas exceto as colunas numéricas e a coluna 'record_id'
df_categoricas = df.drop(columns=[*colunas_numericas, 'record_id'])

# transforma os valores das colunas categóricas em numéricos
df_categoricas['classificacao_tnm_clinico_t'] = df_categoricas['classificacao_tnm_clinico_t'].replace({
    'IS': 0
}).astype(int)

df_categoricas['receptor_de_estrogenio'] = df_categoricas['receptor_de_estrogenio'].replace({
    'negativo': 0,
    'positivo': 1
}).astype(int)

df_categoricas['receptor_de_progesterona'] = df_categoricas['receptor_de_progesterona'].replace({
    'negativo': 0,
    'positivo': 1
}).astype(int)

df_categoricas['ki67_14'] = df_categoricas['ki67_14'].replace({
    'negativo': 0,
    'positivo': 1
}).astype(int)

df_categoricas['classificacao_tnm_clinico_n'] = df_categoricas['classificacao_tnm_clinico_n'].astype(int)

df_categoricas['classificacao_tnm_clinico_m'] = df_categoricas['classificacao_tnm_clinico_m'].astype(int)

df_categoricas['subtipo_tumoral'] = df_categoricas['subtipo_tumoral'].astype(int)

df_categoricas['regime_de_tratamento'] = df_categoricas['regime_de_tratamento'].replace({
    'Terapia Adjuvante': 0,
    'Terapia Neoadjuvante': 1
}).astype(int)

# preenche com zero os valores nulos da coluna `atividade_fisica`
df_categoricas.loc[df_categoricas['atividade_fisica'].isna(), 'atividade_fisica'] = 0
# transforma os valores da coluna `atividade_fisica`
df_categoricas['atividade_fisica'] = df_categoricas['atividade_fisica'].replace({
    'Não pratica': -1,
    'Pratica esporadicamente': 1,
    'Pratica regularmente': 2,
    'Pratica frequentemente': 3
}).astype(int)

df_categoricas = pd.concat([df_categoricas, pd.get_dummies(df_categoricas['estadio_clinico'], prefix='est_c_')], axis=1)
df_categoricas = pd.concat([df_categoricas, pd.get_dummies(df_categoricas['her2_por_ihc'], prefix='her2_')], axis=1)


# Removendo colunas desnecessárias
df_categoricas = normalize_columns_name(df_categoricas).drop(columns=['estadio_clinico', 'her2_por_ihc'])

# Merge categóricas e numéricas
df = pd.concat([df_categoricas, df_numericas], axis=1)

# definindo x e y para separar em treino e teste
df_x = df.drop(columns=['regime_de_tratamento'])

df_x[colunas_numericas] = scaler.transform(df_x[colunas_numericas])

# Realiza as predições
predicoes = rf_classifier.predict(df_x)
# Troca as predições de 0 e 1 para strings
predicoes_em_string = np.where(predicoes == 1, 'neoadjuvante', 'adjuvante')
# Adiciona as predições ao dataframe
df['regime_recomendado'] = predicoes_em_string
# Transforma os dados de volta em csv
df.to_csv('predicoes.csv', encoding = 'utf-8-sig', index=False)
# Download
files.download('/content/predicoes.csv')

SELECIONE O DATAFRAME DE DADOS DEMOGRÁFICOS: 


IndexError: ignored